In [ ]:
# 모델 선택: RandomForestRegressor가 XGBoost나 LightGBM보다 좋은 성능을 보임.

# 최적화: GridSearchCV와 교차 검증을 통해 RandomForestRegressor의 
# 하이퍼파라미터를 최적화

In [7]:
pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 19.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 복합 파생 변수가 적용된 최종 데이터 불러오기
train_df_combo = pd.read_csv('../preprocessed_data/train_processed_combo.csv')
test_df_combo = pd.read_csv('../preprocessed_data/test_processed_combo.csv')

# 피처(X)와 타겟(y) 분리
X = train_df_combo.drop(['ID', 'stress_score'], axis=1)
y = train_df_combo['stress_score']
X_test = test_df_combo.drop('ID', axis=1)

# K-Fold 교차 검증 설정 (5개의 폴드로 나눔)
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))
models = []
rmse_scores = []

# K-Fold 교차 검증
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"Fold {fold+1}/{n_splits} 시작")
    
    # 훈련 데이터와 검증 데이터 분리
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # RandomForestRegressor 모델 초기화 및 학습
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train.values.ravel())
    
    # 검증 데이터 예측
    oof_preds[val_index] = rf_model.predict(X_val)
    
    # 테스트 데이터 예측 (각 폴드마다 예측 후 평균을 내기 위함)
    test_preds += rf_model.predict(X_test) / n_splits
    
    # 모델 성능 평가
    fold_rmse = np.sqrt(mean_squared_error(y_val, oof_preds[val_index]))
    rmse_scores.append(fold_rmse)
    print(f"Fold {fold+1} RMSE: {fold_rmse:.4f}")
    
# 교차 검증 평균 RMSE
mean_rmse = np.mean(rmse_scores)
print(f"\n평균 교차 검증 RMSE: {mean_rmse:.4f}")

# 제출 파일 생성
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['stress_score'] = test_preds
submission_df.to_csv('submission_rf_cv.csv', index=False)

print("교차 검증을 적용한 RandomForest 모델의 제출 파일 'submission_rf_cv.csv'가 성공적으로 생성되었습니다.")

Fold 1/5 시작
Fold 1 RMSE: 0.2333
Fold 2/5 시작
Fold 2 RMSE: 0.2462
Fold 3/5 시작
Fold 3 RMSE: 0.2469
Fold 4/5 시작
Fold 4 RMSE: 0.2497
Fold 5/5 시작
Fold 5 RMSE: 0.2437

평균 교차 검증 RMSE: 0.2440
교차 검증을 적용한 RandomForest 모델의 제출 파일 'submission_rf_cv.csv'가 성공적으로 생성되었습니다.


In [10]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 복합 파생 변수가 적용된 최종 데이터 불러오기
train_df_combo = pd.read_csv('../preprocessed_data/train_processed_combo.csv')
test_df_combo = pd.read_csv('../preprocessed_data/test_processed_combo.csv')

# 피처(X)와 타겟(y) 분리
X = train_df_combo.drop(['ID', 'stress_score'], axis=1)
y = train_df_combo['stress_score']
X_test = test_df_combo.drop('ID', axis=1)

# K-Fold 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# RandomForestRegressor 모델 초기화
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# 튜닝할 하이퍼파라미터 범위 설정 (더 넓게)
param_grid = {
    'n_estimators': [100, 200, 400],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# 최적의 하이퍼파라미터 출력
best_params = grid_search.best_params_
print(f"최적의 하이퍼파라미터: {best_params}")

# 최적의 모델로 최종 예측
best_model = grid_search.best_estimator_
final_predictions_tuned = best_model.predict(X_test)

# 제출 파일 생성
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['stress_score'] = final_predictions_tuned
submission_df.to_csv('submission_final_tuned.csv', index=False)

print("최종 하이퍼파라미터 튜닝이 완료된 모델의 제출 파일 'submission_final_tuned.csv'가 성공적으로 생성되었습니다.")

최적의 하이퍼파라미터: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
최종 하이퍼파라미터 튜닝이 완료된 모델의 제출 파일 'submission_final_tuned.csv'가 성공적으로 생성되었습니다.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 복합 파생 변수가 적용된 최종 데이터 불러오기
train_df_combo = pd.read_csv('../preprocessed_data/train_processed_combo.csv')

# 피처(X)와 타겟(y) 분리
X = train_df_combo.drop(['ID', 'stress_score'], axis=1)
y = train_df_combo['stress_score']

# K-Fold 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# RandomForestRegressor 모델 초기화
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# 튜닝할 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 400],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# 최적의 모델 찾기
best_model = grid_search.best_estimator_

# 특성 중요도 추출
feature_importances = pd.Series(best_model.feature_importances_, index=X.columns)
top_10_features = feature_importances.nlargest(10)
print("\n최고점 모델이 중요하게 생각하는 변수 TOP 10:")
print(top_10_features)